# Federated Learning

This notebook is the simulation notebook adapted so that intermediate data is saved.
This data can be used to unit test the Scala port of the server-side.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import sys
sys.path.insert(0, '..')

In [4]:
from data.frecency import sample, frecency_points
from data.frecency import sample_suggestions_normal as sample_suggestions

In [5]:
from optimizers import GradientDescent, AdaptiveGradientDescent, DecayedGradientDescent, RProp, Adam
from utils import ModelCheckpoint

## Helper functions

In [6]:
def svm_loss(preds, ys, delta=0):
    correct = ys.argmax()
    score_correct = preds[correct]
    
    loss = 0
    
    for i, pred in enumerate(preds):
        loss += max(0, pred + delta - score_correct)            
            
    return loss

In [7]:
def sample_url_features(num_samples):
    frequencies = np.int32(np.random.exponential(7, size=num_samples)) + 1
    frequencies = np.int32(np.ones(num_samples))
    X = []
    
    for frequency in frequencies:
        num_sampled = min(10, frequency)
        features = sample_weighted(num_sampled, weights).sum(axis=0)
        X.append(frequency / num_sampled * features)
        
    return np.array(X)

In [8]:
def rank_accuracy(y, preds):
    correct = 0.
    
    for yi, pi in zip(y, preds):
        if yi[pi.argmax()] == yi.max():
            correct += 1
            
    return correct / len(y)

## Federated Learning

In [9]:
import random

In [10]:
class Server:
    def __init__(self, clients):
        self.clients = clients
        
        num_features = len(frecency_points)
        self.W = np.int32(frecency_points + (np.random.random(size=(num_features)) - 0.5) * 300)
        self.W = np.maximum(1, self.W)
    
    def fit(self, optimizer, num_iterations, num_clients_per_iteration, constraints=[], callbacks=[]):        
        update_list = []
        W_list = [self.W.copy()]
        
        for j in range(num_iterations):
            clients = random.sample(self.clients, num_clients_per_iteration)
            updates, losses = zip(*[client.request_update(self) for client in clients])
            update_list.append(updates)
            
            gradient = np.mean(updates, axis=0)
            loss = np.mean(losses, axis=0)
            
            print("[%d/%d] training loss across clients %.5f" % (j + 1, num_iterations, loss))
            
            for callback in callbacks:
                callback(self)
            
            self.W += np.int32(optimizer(gradient))
            
            for constraint in constraints:
                self.W = constraint(self.W)
                
            W_list.append(self.W.copy())
                
        return update_list, W_list
            
    def predict(self, X):
        preds = []
        
        for x in X:
            scores = x.dot(self.W)
            preds.append(scores)
        
        return preds

In [11]:
class FrecencyConstraints:
    def __call__(self, gradient):
        return gradient - min(0, gradient.min())

## Numerical gradient computation

In [12]:
def full_loss(model, loss_fn, X, y):
    preds = model.predict(X)
    return sum([loss_fn(pi, yi) for pi, yi in zip(preds, y)]) / len(X)

In [13]:
class NumericalClient:
    def __init__(self, data_generator, delta=0):
        self.data_generator = data_generator
        self.delta = 0
    
    def request_update(self, model, eps=1):
        X, y = self.data_generator()
        loss = full_loss(model, svm_loss, X, y)
        
        num_features = X[0].shape[1]
        gradient = []
        
        for i in range(num_features):
            model.W[i] -= eps
            loss1 = full_loss(model, svm_loss, X, y)
            
            model.W[i] += 2 * eps
            loss2 = full_loss(model, svm_loss, X, y)
            
            finite_difference = (loss1 - loss2) / (2 * eps)
            gradient.append(finite_difference)
            
            model.W[i] -= eps
        
        return gradient, loss

## Training

In [14]:
clients = [NumericalClient(lambda: sample_suggestions(np.int32(np.random.exponential(.8)) + 2)) for _ in range(5000)]

In [15]:
np.random.seed(10)
opt = opt = RProp(2., len(frecency_points), min_value=1, max_value=3, alpha=2., beta=0.6)
server = Server(clients)
updates, Ws = server.fit(optimizer=opt,
          num_iterations=30,
           num_clients_per_iteration=400,
           constraints=[FrecencyConstraints()],
          callbacks=[ModelCheckpoint(rank_accuracy, sample_suggestions, 5000)])

[1/30] training loss across clients 148.37488
[ModelCheckpoint] New best model with 0.56180 validation accuracy
[2/30] training loss across clients 147.48179
validation: 0.551 accuracy
[3/30] training loss across clients 135.38525
validation: 0.561 accuracy
[4/30] training loss across clients 113.15793
[ModelCheckpoint] New best model with 0.57460 validation accuracy
[5/30] training loss across clients 113.14079
validation: 0.570 accuracy
[6/30] training loss across clients 94.57138
validation: 0.571 accuracy
[7/30] training loss across clients 95.50639
[ModelCheckpoint] New best model with 0.58480 validation accuracy
[8/30] training loss across clients 87.81560
validation: 0.577 accuracy
[9/30] training loss across clients 82.10962
[ModelCheckpoint] New best model with 0.58980 validation accuracy
[10/30] training loss across clients 75.38465
validation: 0.586 accuracy
[11/30] training loss across clients 64.29371
[ModelCheckpoint] New best model with 0.59180 validation accuracy
[12/30

## Exporting the data

In [16]:
for i in range(len(updates)):
    np.savetxt("updates-%.2d.csv" % i, updates[i], fmt="%.7f")

In [17]:
np.savetxt("weights.csv", np.int32(Ws), fmt="%d")